## Etapa 1: Importação das bibliotecas

In [27]:
import re
import xml.etree.ElementTree as ET
import re

### Bibliotecas locais

In [28]:
from Resources import Categorize

## Etapa 2: Carregando o dicionário

### BlackList

In [29]:
def testStrings(string):
    string = string.lower()
    if(string != 'SIL2007'.lower()
       and string != 'SIL 2007'.lower()
       and string != 'SIL 2009'.lower()
       and string != 'SIL2009'.lower()
       and string != 'sil2009'.lower()
       and string != 'SIL'.lower()
       and string != 'page number'.lower()
       and string != 'number'.lower()
       and string != 'MEL SC GRUPO'.lower()
       and string != 'Daniel'.lower()
       and string != 'mel sc sinais proprios'.lower()
       and string != 'meu sinal'.lower()
       and string != 'MEL SC NT'.lower()
       and string != 'MEL SC NOVOS SINAIS'.lower()
       and string != 'MEL SC SINAIS PRÓPRIOS'.lower()
       and string != 'MEL SC  VT'.lower()
       and string != 'MEL SC GLOSSÁRIO'.lower()
       and string != 'MEL SC '.lower()
       and string != 'MEL SC'.lower()
       and string != 'MEL SC PERSONAGENS BÍBLICOS'.lower()
       and string != 'Kontakt_Symbol'.lower()
       and string != 'Symbol_Proportionen'.lower()
       and string != 'é'.lower()
       and string != 'É'.lower()
       and string != 'cm'.lower()
       and string != 'EAL SC'.lower()
       and string != 'Marcos'.lower()
       and string != 'Gabriel'.lower()
       and string != 'rodrigo'.lower()
       and string != 'Gisele'.lower()
       and string != 'leticia'.lower()
       and string != 'maria'.lower()
       and string != 'nome'.lower()
       and string != 'Aline'.lower()
       and string != 'Maria'.lower()
       and string != 'pedro'.lower()
       and string != 'lucas'.lower()
       and string != 'escrita de sinais'.lower()
       and string != 'sinal'.lower()
       and string != 'ana'.lower()
       and string != ' '.lower()
       and string != 'marco'.lower()
       and string != 'teste'.lower()
       and string != 'libras'.lower()
       and string != 'Jay'.lower()
       and string != 'Kontakt Symbol'.lower()
       and string != 'pantomimische Darstellung'.lower()
       and string != 'Gedankenpause'.lower()
       and string != 'PAM'.lower()
       and string != 'name sign A'.lower()
       and string != 'name sign AB'.lower()
       and string != 'name sign B'.lower()
       and string != 'name sign C R'.lower()
       and string != 'name sign CF'.lower()
       and string != 'name sign CW'.lower()
       and string != 'name sign D'.lower()
       and string != 'name sign E'.lower()
       and string != 'name sign F'.lower()
       and string != 'name sign J'.lower()
       and string != 'name sign K'.lower()
       and string != 'name sign M'.lower()
       and string != 'name sign P'.lower()
       and string != 'name sign S'.lower()
       and string != 'name sign T'.lower()
       and string != 'name sign V'.lower()
       and string != 'name sign W'.lower()
       and string != 'V sign name'.lower()
       and string != 'pronoun'.lower()
       ):
        return True


### Rotina

#### Carrega dicionario e chama a função

In [30]:
def create_dicts(language, type):
    tree = ET.parse('Dicts/{}.spml'.format(language))
    root = tree.getroot()

### Criando o array com os dados brutos

#### Vocabulário artifical com todas as possibilidades

In [31]:
# vocab = r'(S[123][0-9a-f]{2}[0-5][0-9a-f])'
# x = list(exrex.generate(vocab))

In [32]:
fsw_model = r'((A(S[123][0-9a-f]{2}[0-5][0-9a-f])+)|S[0-9a-fA-F]{3}[0-5]{1}[0-9a-fA-F]{1}[0-9]{3}x[0-9]{3})|([BLMR][0-9]{3}x[0-9]{3})|(S[0-9a-fA-F]{3}[0-5]{1}[0-9a-fA-F]{1})'


#### Vocabulários Slim

In [33]:
def slim(language):
    
    tree = ET.parse('Dicts/{}.spml'.format(language))
    root = tree.getroot()
        
    palavras = []
    txt_file = 'Outputs/Dicts/{}_raw_slim.txt'.format(language)
    raw = open(txt_file, 'w')

    for item in root:
        if(item.tag == 'entry'):
            glosas = []
            for subitem in item:
                if(subitem.tag == 'term'):
                    if(re.match(fsw_model, subitem.text)):
                        cod = subitem.text
                    else:
                        if(testStrings(subitem.text)):
                            glosas.append(Categorize.clean_text(subitem.text))
            if (len(glosas) > 0 and len(cod) > 0):
                palavras.append(glosas)
                for g in glosas:
                    detalhe = Categorize.details(cod)
                    combined = ""
                    for det in detalhe:
                        combined += "{:} ".format(det['full'])

                    if(det['base'] != 'punctuation'):
                        raw.write('{}&{}'.format(g, combined))
                        raw.write('\n')
    raw.close()


#### Modificação Alfa

In [34]:
def alfa(language):

    tree = ET.parse('Dicts/{}.spml'.format(language))
    root = tree.getroot()

    txt_file = 'Outputs/Dicts/{}_raw_alfa.txt'.format(language)
    raw = open(txt_file, 'w')
    for item in root:
        if(item.tag == 'entry'):
            glosas = []
            for subitem in item:
                if(subitem.tag == 'term'):
                    if(re.match(fsw_model, subitem.text)):
                        cod = subitem.text
                    else:
                        if(testStrings(subitem.text)):
                            glosas.append(Categorize.clean_text(subitem.text))
            if (len(glosas) > 0 and len(cod) > 0):
                for g in glosas:
                    detalhe = Categorize.details(cod)
                    combined = ""
                    left = 0
                    right = 0
                    count_right = 0
                    count_left = 0
                    base_right = None
                    base_left = None

                    for det in detalhe:
                        lr = det['left_right'] if det['left_right'] else ''
                        combined += "{:} {:} {:} {:}. ".format(
                            det['N0'], lr, det['base'], det['full'])

                        if (base_right != det['base'] and det['N0'] == 'hand' and det['left_right'] == 'right'):
                            count_right += 1
                        if (base_left != det['base'] and det['N0'] == 'hand' and det['left_right'] == 'left'):
                            count_left += 1

                        if (det['N0'] == 'hand' and det['left_right'] == 'right'):
                            right = 1
                            base_right = det['base']

                        if (det['N0'] == 'hand' and det['left_right'] == 'left'):
                            left = 1
                            base_left = det['base']
                    combined += "estaticidadeR{:}. ".format(
                        str(True if (count_right < 2) else False))
                    combined += "estaticidadeL{:}. ".format(
                        str(True if (count_left < 2) else False))
                    combined += "hands_qtd{:}. ".format(str(right+left))

                    if(det['base'] != 'punctuation'):
                        raw.write('{}&{}'.format(g, combined))
                        raw.write('\n')
    raw.close()


#### Detalhado

In [35]:
def details(language):
    tree = ET.parse('Dicts/{}.spml'.format(language))
    root = tree.getroot()

    txt_file = 'Outputs/Dicts/{}_raw_details.txt'.format(language)
    raw = open(txt_file, 'w')
    for item in root:
        if(item.tag == 'entry'):
            glosas = []
            for subitem in item:
                if(subitem.tag == 'term'):
                    if(re.match(fsw_model, subitem.text)):
                        cod = subitem.text
                    else:
                        if(testStrings(subitem.text)):
                            glosas.append(Categorize.clean_text(subitem.text))
            if (len(glosas) > 0 and len(cod) > 0):
                for g in glosas:
                    detalhe = Categorize.details(cod)
                    combined = ""
                    left = 0
                    right = 0
                    count_right = 0
                    count_left = 0
                    base_right = None
                    base_left = None

                    for det in detalhe:
                        lr = det['left_right'] if det['left_right'] else ''
                        combined += "{:} {:} {:} {:}{:} {:}. ".format(
                            det['N0'], lr, det['base'], det['base'], det['N2'], det['full'])

                        if (base_right != det['base'] and det['N0'] == 'hand' and det['left_right'] == 'right'):
                            count_right += 1
                        if (base_left != det['base'] and det['N0'] == 'hand' and det['left_right'] == 'left'):
                            count_left += 1

                        if (det['N0'] == 'hand' and det['left_right'] == 'right'):
                            right = 1
                            base_right = det['base']

                        if (det['N0'] == 'hand' and det['left_right'] == 'left'):
                            left = 1
                            base_left = det['base']
                    combined += "estaticidadeR{:}. ".format(
                        str(True if (count_right < 2) else False))
                    combined += "estaticidadeL{:}. ".format(
                        str(True if (count_left < 2) else False))
                    combined += "hands_qtd{:}. ".format(str(right+left))

                    if(det['base'] != 'punctuation'):
                        raw.write('{}&{}'.format(g, combined))
                        raw.write('\n')
    raw.close()


In [36]:
dicts = ['asl','germany','libras']

In [37]:
for language in dicts:
    slim(language)
    alfa(language)
    details(language)